In [87]:
import logging
import pandas as pd
import json


In [124]:
def openFile(file_path, mode='r'):
    try:
        return open(file_path, mode)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        raise

def getDataframeFromFile(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('|')
        
        item = {}

        if len(rec) > 6:
            item['movieId'] = rec[0]
            item['name'] = rec[1]
            item['release_date'] = rec[2]
            item['url'] = rec[4]

            #5 => 23
            item['genre'] = [genre[idx]['name'] for  idx, g in enumerate(range(5,24)) if rec[g] == '1']

        elif len(rec) == 2:
            item['id'] = rec[1]
            item['name'] = rec[0]
            
        elif len(rec) == 5:
            item['userId'] = rec[0]
            item['age'] = rec[1]
            item['gender'] = rec[2]
            item['function'] = rec[3]
        else:
            print("Invalid record format. Skipping.")
            return

        data.append(item)
    filed.close()
    return data

def getDataframe(filed):
    data = []
    while True:
        line = filed.readline()
        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('\t')

        item = {}
        item['userId'] = rec[0]
        item['movieId'] = rec[1]
        item['rating'] = rec[2]
        item['timestamp'] = rec[3]
        data.append(item)
    filed.close()
    return data

#=>Data
fileGenre = openFile('../data/u.genre')
fileItem = openFile('../data/u.item')
fileUser = openFile('../data/u.user')
fileData = openFile('../data/u.data')

df_relationship_data = pd.DataFrame(getDataframe(fileData))

genre = getDataframeFromFile(fileGenre)
items = getDataframeFromFile(fileItem)
users = getDataframeFromFile(fileUser)

df_movies = pd.DataFrame(items)
df_users = pd.DataFrame(users)



def tojson(fileName,df):
    data = {}
    data["results"] = df.to_dict(orient='records')
    data["count"] = len(df)
    with open(f'../data/json/{fileName}.json','w') as outputFile:
        json.dump(data,outputFile)


tojson('movies',df_movies)
tojson('users',df_users)
tojson('ratings',df_relationship_data)

# moviesData["results"]

# df_relationship_data.groupby('userId').apply(lambda x:print(x))

      userId movieId rating  timestamp
202        1      61      4  878542420
305        1     189      3  888732928
333        1      33      4  878542699
334        1     160      4  875072547
478        1      20      4  887431883
...      ...     ...    ...        ...
92049      1      28      4  875072173
92487      1     172      5  874965478
94019      1     122      3  875241498
96699      1     152      5  878542589
99073      1      94      2  875072956

[272 rows x 4 columns]
      userId movieId rating  timestamp
40        10      16      4  877888877
158       10     486      4  877886846
386       10     175      3  877888677
544       10     611      5  877886722
606       10       7      4  877892210
...      ...     ...    ...        ...
94371     10     686      4  877886911
96580     10     604      4  877892110
97430     10      93      4  877892160
99578     10       9      4  877889005
99846     10     198      3  877889005

[184 rows x 4 columns]
      userId mov

""
